<a href="https://colab.research.google.com/github/JimKing100/nfl-test/blob/master/Rookie_Offense_Calc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Imports
import pandas as pd

In [0]:
# Load the raw data
player_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-revised/rookies_non_offense.csv')
offense_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-revised/offense.csv')
td_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-revised/td.csv')
game_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-revised/game.csv')

In [3]:
# The dataframe of all 2019 non-rookie offensive players and their rookie year
player_df.head()

,player,first,last,name,position1,start
0,TB-2300,Tom,Brady,Tom Brady,QB,2000
1,DB-3800,Drew,Brees,Drew Brees,QB,2001
2,JM-2900,Josh,McCown,Josh McCown,QB,2002
3,BR-1100,Ben,Roethlisberger,Ben Roethlisberger,QB,2004
4,EM-0200,Eli,Manning,Eli Manning,QB,2004


In [4]:
# The offense dataframe - raw offensive data for all 2019 QBs, RBs, WRs, TEs for seasons 2000-2019
offense_df.head()

,uid,gid,week,player,del,first,last,name,position1,pa,pc,py,ints,tdp,ra,sra,ry,tdr,trg,rec,recy,tdrec,ret,rety,tdret,fuml,peny,conv,snp,fp,fp2,fp3,game,seas,year,team
0,3599,175,13,TB-2300,KEEP,Tom,Brady,Tom Brady,QB,3,1,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.24,0.24,0.24,0,1,2000,NE
1,5841,284,2,TB-2300,KEEP,Tom,Brady,Tom Brady,QB,10,5,46,0,0,1,1,9,0,0,0,0,0,0,0,0,0,0,0,0,2.74,2.74,2.74,0,2,2001,NE
2,6050,295,3,TB-2300,KEEP,Tom,Brady,Tom Brady,QB,23,13,168,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,6.92,6.92,6.92,0,2,2001,NE
3,6330,308,4,TB-2300,KEEP,Tom,Brady,Tom Brady,QB,24,12,86,0,0,2,1,9,0,0,0,0,0,0,0,0,2,0,0,0,0.34,0.34,2.34,0,2,2001,NE
4,6672,325,5,TB-2300,KEEP,Tom,Brady,Tom Brady,QB,54,33,364,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22.56,22.56,25.56,0,2,2001,NE


In [5]:
# The touchdown dataframe - raw touchdown data for rushing, receiving and passing TD plays of 40 yards and greater.
td_df.head()

,pid,psr,first_qb,last_qb,name_qb,gid,year,week,qtr,min,sec,dwn,yds,pts,player,first,last,name,type
0,75,CC-1400,Chris,Chandler,Chris Chandler,1,2000,1,2,4,36,2,44,7,TM-0900,Terance,Mathis,Terance Mathis,REC
1,98,CC-1400,Chris,Chandler,Chris Chandler,1,2000,1,3,15,0,1,48,7,SJ-0400,Shawn,Jefferson,Shawn Jefferson,REC
2,1051,CM-2200,Cade,McNown,Cade McNown,7,2000,1,3,15,0,3,48,7,MR-1400,Marcus,Robinson,Marcus Robinson,REC
3,1509,KC-2000,Kerry,Collins,Kerry Collins,10,2000,1,2,10,11,1,78,7,TB-0400,Tiki,Barber,Tiki Barber,RUSH
4,1819,TB-0200,Tony,Banks,Tony Banks,12,2000,1,1,4,13,1,53,7,QI-0100,Qadry,Ismail,Qadry Ismail,REC


In [6]:
# The game dataframe - raw game data to calculate bye weeks
game_df.head()

,gid,seas,wk,day,v,h,stad,temp,humd,wspd,wdir,cond,surf,ou,sprv,ptsv,ptsh
0,1,2000,1,SUN,SF,ATL,Georgia Dome,79.0,NaN,NaN,NaN,Dome,AstroTurf,42.5,7.0,28,36
1,2,2000,1,SUN,JAC,CLE,Cleveland Browns Stadium,78.0,63.0,9.0,NE,Sunny,Grass,38.0,-10.0,27,7
2,3,2000,1,SUN,PHI,DAL,Texas Stadium,109.0,19.0,5.0,S,Sunny,AstroTurf,40.0,6.0,41,14
3,4,2000,1,SUN,NYJ,GB,Lambeau Field,77.0,66.0,5.0,E,Mostly Cloudy,Grass,36.0,2.5,20,16
4,5,2000,1,SUN,IND,KC,Arrowhead Stadium,90.0,50.0,8.0,E,Mostly Sunny,Grass,44.0,-3.0,27,14


In [0]:
# Pull the stats to calculate the player points
def pull_stats(df, df1, df2, col, p, y, g):

    if (df[col].loc[(df['name']==p) & (df['year']==y) & (df['week']==g)].empty):
        start_year = df1['start'].loc[(df1['name']==p)].iloc[0]
        if y < start_year:
            points = float('NaN')
        else:
            points = 0
    else:
        rush_tds = df['tdr'].loc[(df['name']==p) & (df['year']==y) & (df['week']==g)].iloc[0]
        receive_tds = df['tdrec'].loc[(df['name']==p) & (df['year']==y) & (df['week']==g)].iloc[0]
        pass_tds = df['tdp'].loc[(df['name']==p) & (df['year']==y) & (df['week']==g)].iloc[0]
        return_tds = df['tdret'].loc[(df['name']==p) & (df['year']==y) & (df['week']==g)].iloc[0] 
        convs = df['conv'].loc[(df['name']==p) & (df['year']==y) & (df['week']==g)].iloc[0]
        rush_yds = df['ry'].loc[(df['name']==p) & (df['year']==y) & (df['week']==g)].iloc[0]
        receive_yds = df['recy'].loc[(df['name']==p) & (df['year']==y) & (df['week']==g)].iloc[0]
        pass_yds = df['py'].loc[(df['name']==p) & (df['year']==y) & (df['week']==g)].iloc[0]
        ints = df['ints'].loc[(df['name']==p) & (df['year']==y) & (df['week']==g)].iloc[0]
        fumbles = df['fuml'].loc[(df['name']==p) & (df['year']==y) & (df['week']==g)].iloc[0]
        rush_rec_td40 = df2['name'].loc[(df2['name']==p) & (df2['year']==y) & (df2['week']==g)].count()
        pass_td40 = df2['name_qb'].loc[(df2['name_qb']==p) & (df2['year']==y) & (df2['week']==g)].count()
        

        points = (rush_tds * 6) + \
                 (receive_tds * 6) + \
                 (pass_tds * 4) + \
                 (return_tds * 6) + \
                 (convs * 2) + \
                 (rush_yds / 10) + \
                 (receive_yds /10) + \
                 (pass_yds / 25) - \
                 (ints * 2) - \
                 (fumbles * 2) + \
                 (rush_rec_td40 * 2) + \
                 (pass_td40)

        #print('rush tds', rush_tds, 'rec tds', receive_tds, 'pass tds', pass_tds, 
        #      'return tds', return_tds, 'conversions', convs, 'rush yds', rush_yds, 
        #      'receive yds', receive_yds, 'pass yds', pass_yds, 'ints', ints, 'fumbles', fumbles,
        #       'rush/rec TD 40', rush_rec_td40, 'pass TD 40', pass_td40)

    return points

In [0]:
# Add a row to the final_df dataframe
# Each row represents the actual offensive points for each game for each offensive player
def add_row(df, p, f, l, n, pos, value_list):

    df = df.append({'player': p,
                    'first': f,
                    'last': l,
                    'name': n,
                    'position1': pos,
                    'game1': value_list[0], 
                    'game2': value_list[1],
                    'game3': value_list[2], 
                    'game4': value_list[3], 
                    'game5': value_list[4], 
                    'game6': value_list[5], 
                    'game7': value_list[6], 
                    'game8': value_list[7], 
                    'game9': value_list[8],
                    'game10': value_list[9],
                    'game11': value_list[10],
                    'game12': value_list[11],
                    'game13': value_list[12],
                    'game14': value_list[13],
                    'game15': value_list[14],
                    'game16': value_list[15]
                    }, ignore_index=True)
    
    return df

In [9]:
# The main code for iterating through the kicker list, calculating the points and adding the rows
# to the final_df dataframe.
column_names = ['player',
                'first',
                'last',
                'name',
                'position1', 
                'game1',
                'game2',
                'game3', 
                'game4', 
                'game5', 
                'game6', 
                'game7', 
                'game8', 
                'game9',
                'game10',
                'game11',
                'game12',
                'game13',
                'game14',
                'game15',
                'game16'
                ]

oplayers_df = player_df.loc[(player_df['position1'] == 'QB') |
                            (player_df['position1'] == 'RB') |
                            (player_df['position1'] == 'WR') |
                            (player_df['position1'] == 'TE')]
player_list = oplayers_df['name'].tolist()

final_df = pd.DataFrame(columns = column_names)

for player in player_list:

    first = player_df['first'].loc[(player_df['name']==player)].iloc[0]
    last = player_df['last'].loc[(player_df['name']==player)].iloc[0]
    name = player_df['name'].loc[(player_df['name']==player)].iloc[0]
    position1 = player_df['position1'].loc[(player_df['name']==player)].iloc[0]
    year = player_df['start'].loc[(player_df['name']==player)].iloc[0]

    print(player)

    for s in range(1, 17):
        if offense_df['team'].loc[(offense_df['name']==player) & (offense_df['seas']==s)].empty:
            x = 0
        else:
            team = offense_df['team'].loc[(offense_df['name']==player) & (offense_df['seas']==s)].iloc[0]
            break
    new_team = team

    player_scores = []

 
    week = 0
    for game in range(1,17):
        week = week + 1

        if (offense_df['team'].loc[(offense_df['name']==player) & (offense_df['year']==year) & (offense_df['week']==week)].empty):
            x = 0
        else:
            new_team = offense_df['team'].loc[(offense_df['name']==player) & (offense_df['year']==year) & (offense_df['week']==week)].iloc[0]
            
        if team != new_team:
            team = new_team

        if ((game_df['h'].loc[(game_df['seas']==year) & (game_df['wk']==week) & ((game_df['v']==team) | (game_df['h']==team))].empty) &
            (game_df['v'].loc[(game_df['seas']==year) & (game_df['wk']==week) & ((game_df['v']==team) | (game_df['h']==team))].empty)):
            week = week + 1
            score = pull_stats(offense_df, player_df, td_df, 'fp', player, year, week)
        else:
            score = pull_stats(offense_df, player_df, td_df, 'fp', player, year, week)

        player_scores.append(score)

    final_df = add_row(final_df, player, first, last, name, position1, player_scores)

Tom Brady
Drew Brees
Josh McCown
Ben Roethlisberger
Eli Manning
Matt Schaub
Philip Rivers
Aaron Rodgers
Ryan Fitzpatrick
Matt Moore
Joe Flacco
Matt Ryan
Brian Hoyer
Chase Daniel
Matthew Stafford
Colt McCoy
Andy Dalton
Cam Newton
Tyrod Taylor
Alex Tanney
Case Keenum
Kirk Cousins
Nick Foles
Robert Griffin
Ryan Tannehill
Russell Wilson
Matt Barkley
Mike Glennon
Ryan Griffin
A.J. McCarron
Blake Bortles
Derek Carr
Garrett Gilbert
Jimmy Garoppolo
Teddy Bridgewater
Brett Hundley
Jameis Winston
Marcus Mariota
Sean Mannion
Trevor Siemian
Brandon Allen
Carson Wentz
Dak Prescott
Jacoby Brissett
Jeff Driskel
Jared Goff
Deshaun Watson
Mitchell Trubisky
Nick Mullens
Patrick Mahomes
Taysom Hill
Baker Mayfield
Josh Allen
Josh Rosen
Kyle Allen
Luke Falk
Lamar Jackson
Mason Rudolph
Sam Darnold
Tim Boyle
Darren Sproles
Frank Gore
Adrian Peterson
Marshawn Lynch
LeSean McCoy
James Develin
Anthony Sherman
Bilal Powell
Dion Lewis
Mark Ingram
Nick Bellore
Patrick DiMarco
Taiwan Jones
Alfred Morris
Brandon Bol

In [10]:
final_df.head(50)

,player,first,last,name,position1,game1,game2,game3,game4,game5,game6,game7,game8,game9,game10,game11,game12,game13,game14,game15,game16
0,Tom Brady,Tom,Brady,Tom Brady,QB,0,0,0,0,0,0,0,0,0,0,0,0.24,0,0,0,0
1,Drew Brees,Drew,Brees,Drew Brees,QB,0,0,0,0,0,0,0,14.64,0,0,0,0.00,0,0,0,0
2,Josh McCown,Josh,McCown,Josh McCown,QB,0,0,0,0,0,0,0,0,0,0,0,-0.20,0,0,0,-1.16
3,Ben Roethlisberger,Ben,Roethlisberger,Ben Roethlisberger,QB,0,11.04,8.72,11.16,18.54,16.52,17.14,12.32,7.16,9.12,5.64,20.84,1.56,12.54,15.94,0
4,Eli Manning,Eli,Manning,Eli Manning,QB,2.64,0,0,0,0,0,0,0,0,6.38,3.72,4.52,-4.92,15.08,6.04,15.76
5,Matt Schaub,Matt,Schaub,Matt Schaub,QB,0,0,0,-0.2,0,0,0.36,0,0,0,0,-2.00,0,0,5.62,8.02
6,Philip Rivers,Philip,Rivers,Philip Rivers,QB,0,0,0,0,0,0,0,0,-0.4,0,0,0.00,0,0,0,5.22
7,Aaron Rodgers,Aaron,Rodgers,Aaron Rodgers,QB,0,0,0,0,0,0,0,0,0,0,0,0.00,0,-2.6,0,-0.1
8,Ryan Fitzpatrick,Ryan,Fitzpatrick,Ryan Fitzpatrick,QB,0,0,0,0,0,0,0,0,0,0,26.7,11.72,7,5.06,0,0
9,Matt Moore,Matt,Moore,Matt Moore,QB,0,0,0,0,1.72,0,0,-0.56,-2,0,0.64,0.00,0.84,8.82,9.28,12.96


In [0]:
# Save the results to .csv file
final_df.to_csv('/content/actuals_rookie_offense.csv', index=False)